# League Sports Analysis
## Description
In this, you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport, answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!


## 1
For this, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def res_nhl():

    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df[nhl_df['PTS'] != nhl_df['PTS%']]    #removing division rows
    nhl_df['team'] = nhl_df.apply(lambda x: re.sub('\*','',x['team']), axis=1)
    nhl_df['team'] = nhl_df.apply(lambda x: x['team'].split(' ')[-1], axis=1)
    nhl_df['W/L Ratio'] = nhl_df.apply(lambda x: int(x['W'])/(int(x['W'])+int(x['L'])), axis=1)
    nhl_df = nhl_df.set_index('team')

    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]       #selecting relevant columns
    cities['NHL'] = cities.apply(lambda x: re.sub('\[.*\]','',x['NHL']), axis=1)
    cities['NHL'] = cities.apply(lambda x: x['NHL'].split(' ') if ' ' in x['NHL'] else x['NHL'], axis=1)
    cities = cities.explode('NHL')
    cities = cities.set_index('NHL')

    res = pd.merge(nhl_df, cities, how='inner', left_index=True, right_index=True)
    res['population'] = res.apply(lambda x: int(x['Population (2016 est.)[8]']), axis=1)
    temp = res.groupby('Metropolitan area').agg({'W/L Ratio':np.mean, 'population':np.mean})
    
    return temp

def nhl_correlation():
    
    temp = res_nhl()
        
    population_by_region = temp['population'] # pass in metropolitan area population from cities
    win_loss_by_region = temp['W/L Ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [2]:
nhl_correlation()

0.0608128037551192

## 2
For this, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def res_nba():

    nba_df=pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df.apply(lambda x: re.findall('[a-zA-Z0-9 ]*',x['team'])[0], axis=1)
    nba_df['team'] = nba_df.apply(lambda x: x['team'].split(' ')[-1], axis=1)
    nba_df = nba_df.set_index('team')

    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['NBA'] = cities.apply(lambda x: re.sub('\[.*\]','',x['NBA']), axis=1)
    cities['NBA'] = cities.apply(lambda x: x['NBA'].split(' ') if ' ' in x['NBA'] else x['NBA'], axis=1)
    cities = cities.explode('NBA')
    cities = cities.set_index('NBA')

    res = pd.merge(nba_df, cities, how='inner', left_index=True, right_index=True)
    res['population'] = res.apply(lambda x: int(x['Population (2016 est.)[8]']), axis=1)
    res['W/L%'] = res.apply(lambda x: float(x['W/L%']), axis=1)
    temp = res.groupby('Metropolitan area').agg({'W/L%':np.mean, 'population':np.mean})
    
    return temp

def nba_correlation():
    
    temp = res_nba()
    
    population_by_region = temp['population'] # pass in metropolitan area population from cities
    win_loss_by_region = temp['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [4]:
nba_correlation()

-0.0052899453820452694

## 3
For this, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def res_mlb():

    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df['team'][0] = 'Rsox'      #handling similar last names of teams Red Sox and White Sox
    mlb_df['team'][8] = 'Wsox'
    mlb_df['team'] = mlb_df.apply(lambda x: x['team'].split(' ')[-1], axis=1)
    mlb_df = mlb_df.set_index('team')

    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['MLB'] = cities.apply(lambda x: re.sub('\[.*\]','',x['MLB']), axis=1)
    cities['MLB'][3] = 'Cubs Wsox'
    cities['MLB'][7] = 'Rsox'
    cities['MLB'] = cities.apply(lambda x: x['MLB'].split(' ') if ' ' in x['MLB'] else x['MLB'], axis=1)
    cities = cities.explode('MLB')
    cities = cities.set_index('MLB')

    res = pd.merge(mlb_df, cities, how='inner', left_index=True, right_index=True)
    res['population'] = res.apply(lambda x: int(x['Population (2016 est.)[8]']), axis=1)
    res['W-L%'] = res.apply(lambda x: float(x['W-L%']), axis=1)
    temp = res.groupby('Metropolitan area').agg({'W-L%':np.mean, 'population':np.mean})
    
    return temp

def mlb_correlation():
    
    temp = res_mlb()
    
    population_by_region = temp['population'] # pass in metropolitan area population from cities
    win_loss_by_region = temp['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [6]:
mlb_correlation()

C:\Users\shiva\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0.041413202736278604

## 4
For this, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def res_nfl():
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    nfl_df = nfl_df[(nfl_df['year'] == 2018) & (nfl_df['W'] != nfl_df['W-L%'])]
    nfl_df['team'] = nfl_df.apply(lambda x: re.findall('[a-zA-Z0-9 ]*',x['team'])[0], axis=1)
    nfl_df['team'] = nfl_df.apply(lambda x: x['team'].split(' ')[-1], axis=1)
    nfl_df = nfl_df.set_index('team')

    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities['NFL'] = cities.apply(lambda x: re.sub('\[.*\]','',x['NFL']), axis=1)
    cities['NFL'] = cities.apply(lambda x: x['NFL'].split(' ') if ' ' in x['NFL'] else x['NFL'], axis=1)
    cities = cities.explode('NFL')
    cities = cities.set_index('NFL')

    res = pd.merge(nfl_df, cities, how='inner', left_index=True, right_index=True)
    res['population'] = res.apply(lambda x: int(x['Population (2016 est.)[8]']), axis=1)
    res['W-L%'] = res.apply(lambda x: float(x['W-L%']), axis=1)
    temp = res.groupby('Metropolitan area').agg({'W-L%':np.mean, 'population':np.mean})
    
    return temp

def nfl_correlation():
    
    temp = res_nfl()
    
    population_by_region = temp['population'] # pass in metropolitan area population from cities
    win_loss_by_region = temp['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [8]:
nfl_correlation()

0.2134045407290009

## 5
In this, explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**; With a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from scipy.stats import ttest_rel

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    
    nfl = res_nfl()
    nfl.rename(columns={'W-L%':'WL NFL'}, inplace=True)
    nhl = res_nhl()
    nhl.rename(columns={'W/L Ratio':'WL NHL'}, inplace=True)
    nba = res_nba()
    nba.rename(columns={'W/L%':'WL NBA'}, inplace=True)
    mlb = res_mlb()
    mlb.rename(columns={'W-L%':'WL MLB'}, inplace=True)

    nfl_nhl = pd.merge(nfl, nhl, how='inner', left_index=True, right_index=True)
    nfl_nba = pd.merge(nfl, nba, how='inner', left_index=True, right_index=True)
    nfl_mlb = pd.merge(nfl, mlb, how='inner', left_index=True, right_index=True)
    nhl_nba = pd.merge(nhl, nba, how='inner', left_index=True, right_index=True)
    nhl_mlb = pd.merge(nhl, mlb, how='inner', left_index=True, right_index=True)
    nba_mlb = pd.merge(nba, mlb, how='inner', left_index=True, right_index=True)

    p_nfl_nhl = ttest_rel(nfl_nhl['WL NFL'],nfl_nhl['WL NHL'])[-1]
    p_nfl_nba = ttest_rel(nfl_nba['WL NFL'],nfl_nba['WL NBA'])[-1]
    p_nfl_mlb = ttest_rel(nfl_mlb['WL NFL'],nfl_mlb['WL MLB'])[-1]
    p_nhl_nba = ttest_rel(nhl_nba['WL NHL'],nhl_nba['WL NBA'])[-1]
    p_nhl_mlb = ttest_rel(nhl_mlb['WL NHL'],nhl_mlb['WL MLB'])[-1]
    p_nba_mlb = ttest_rel(nba_mlb['WL NBA'],nba_mlb['WL MLB'])[-1]

    data = [[np.nan, p_nfl_nba, p_nfl_nhl, p_nfl_mlb],[p_nfl_nba, np.nan, p_nhl_nba, p_nba_mlb],[p_nfl_nhl, p_nhl_nba, np.nan, p_nhl_mlb],[p_nfl_mlb, p_nba_mlb, p_nhl_mlb, np.nan]]

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame(data, index=sports, columns=sports)
    p_values
    
    return p_values

In [10]:
sports_team_performance()

C:\Users\shiva\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,NFL,NBA,NHL,MLB
NFL,NaN,0.692438,0.053925,0.524692
NBA,0.692438,NaN,0.087494,0.844946
NHL,0.053925,0.087494,NaN,0.001753
MLB,0.524692,0.844946,0.001753,NaN
